In [2]:
import os
import sys
sys.path.append('../../Project_Clustering')

import numpy as np
import pandas as pd
import pickle 
from matplotlib import cm
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
import myClusterPackage as myPack
from sklearn.mixture import GaussianMixture
from mySpectralClustering import spectralFeatures
from multiprocessing import Pool

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import time

## Extracting point cloud features

In [ ]:
#face_db = []
#face_list = []
#labels = []


#data_folder = os.listdir("../3DFACEPAPER/Data/") # list of all subfolder within Data folder
#data_folder.remove('.DS_Store') 


#for k, expressions in enumerate (data_folder): # will loop within the main folder
   
    # Accessing each subfolder (expression) within the data folder  
#    path =  "../3DFACEPAPER/Data/" + expressions +"/"
#    face_names = os.listdir(path)    # list of all file names within subfolder
    
   
#    for face in face_names: # will loop within the subfolder
        
#        if not face.startswith('.'):
       # import the face and face labels
#            file_name = path + face
        
#            df_face = pd.read_csv(file_name)
#            face_list.append (df_face.values)
#            labels.append (expressions)
            

#pickle.dump(face_list, open('face_data_full','wb'))
#pickle.dump (labels, open('face_labels_full', 'wb'))


    

In [2]:
face_list = pickle.load(open('face_data_full', 'rb'))
face_labels = pickle.load(open('face_labels_full', 'rb'))

all_patch_data = []
n_face = len(face_list)

In [ ]:
#%matplotlib tk


#fig02 = plt.figure(figsize=(15,8))
#ax = Axes3D(fig02)   
#print (face_list[40])
#for k in range (1):
#    mvpk = np.where(face_list[k][:,2]==max(face_list[k][:,2]))
    #mvpk = 
#    ax.scatter(face_list[k][:,0]-face_list[k][mvpk,0], 
#               face_list[k][:,1]-face_list[k][mvpk,1],
#              face_list[k][:,2]- face_list[k][mvpk,2])

#km01 = KMeans(
#        n_clusters=8, init='k-means++',
#        n_init=10, max_iter=300, 
 #       tol=1e-04, random_state=10
 #   )

#mvpk = np.where(face_list[0][:,2]== max(face_list[0][:,2]))
#faceo = face_list[0] - face_list[0][mvpk]

#transFeature = spectralFeatures (face_list,n_dim = 3, n_neighbor = 10)
#df_3Col = transFeature.values    
    
#km01 = km01.fit(df_3Col)
#init_centroids = km01.cluster_centers_
#ax.scatter(init_centroids[:,0], init_centroids[:,1], init_centroids[:,2],'r', s=1000)

In [ ]:
#pool = Pool()

#pool.imap(Fun, product(xrange(N), xrange(N))), chunksize
km = KMeans(
        n_clusters=8, init='k-means++',
        n_init=1, max_iter=300, 
        tol=1e-04, random_state=10
    )



    

  #  mvpk = np.where(face[:,2]== max(face[:,2]))
    
 #   face = face - face[mvpk[0][0]]
#def specFeature (face, km):
for i,face in enumerate(face_list):
    


    mvpk = np.where(face[:,2]== max(face[:,2]))
    
    face = face - face[mvpk[0][0]]

    total_features = []
    
    
    transFeature = spectralFeatures (face, n_dim = 3, n_neighbor = 10)
    df_3Col = transFeature.values
    
    segments = km.fit_predict(df_3Col)
    n_patch = len(np.unique(segments)) 
  #  centroids, _ = myPack.get_centroids(face, segments)
    total_features = []
   
    for j in range (n_patch):
        
        patch_features = []
        clusterLabels = []
        featureLabels = []
        
        patch = face[segments == j]
        
        delta_patch = patch #- centroids [j] 
        
  #      transFeature = spectralFeatures (delta_patch,n_dim = 2, n_neighbor = 10)
   #     df_2Col = transFeature.values
    
        
        ## Applying GMM on each patch
        gmm = GaussianMixture(n_components = 3, covariance_type= 'diag',random_state=10)
        patch_gmm = gmm.fit(delta_patch)
        patch_w = patch_gmm.weights_
        patch_mu = patch_gmm.means_
        patch_sigma = patch_gmm.covariances_
        
        ## creating data frame for testing
   #     n_features = len(patch_w) + len(patch_mu)*len(patch_mu[1]) #+ len(patch_sigma)*len(patch_sigma[1])
    
        # Features extracted from a single patch
        # 3 + 6 + 6 --> repeated across 8 patches
        patch_features = list(patch_w) + list(patch_mu.ravel())+ list(patch_sigma.ravel())
        
        # concatenating from all patches
        total_features = total_features + patch_features
    # adding features from all patches to the face database 

    
df_3D_face = pd.DataFrame({'Patch_Features':all_patch_data,'Classes':face_labels})                                      

# Encoding class labels 
le = preprocessing.LabelEncoder() 
df_3D_face['Labels']= le.fit_transform(df_3D_face['Classes']) 
 

In [ ]:
pool = Pool(8)

#pool.imap(Fun, product(xrange(N), xrange(N))), chunksize

for i,face in enumerate(pool.starmap(face_list)):
    
    
    km = KMeans(
        n_clusters=8, init='k-means++',
        n_init=1, max_iter=300, 
        tol=1e-04, random_state=10
    )

  #  mvpk = np.where(face[:,2]== max(face[:,2]))
    
 #   face = face - face[mvpk[0][0]]
    
    transFeature = spectralFeatures (face, n_dim = 3, n_neighbor = 10)
    df_3Col = transFeature.values
    
    segments = km.fit_predict(df_3Col)
    n_patch = len(np.unique(segments)) 
    centroids, _ = myPack.get_centroids(face, segments)
    total_features = []
   
    for j in range (n_patch):
        
        patch_features = []
        clusterLabels = []
        featureLabels = []
        
        patch = face[segments == j]
        
        delta_patch = patch - centroids [j] 
        
  #      transFeature = spectralFeatures (delta_patch,n_dim = 2, n_neighbor = 10)
   #     df_2Col = transFeature.values
    
        
        ## Applying GMM on each patch
        gmm = GaussianMixture(n_components = 3, covariance_type= 'diag',random_state=10)
        patch_gmm = gmm.fit(delta_patch)
        patch_w = patch_gmm.weights_
        patch_mu = patch_gmm.means_
        patch_sigma = patch_gmm.covariances_
        
        ## creating data frame for testing
   #     n_features = len(patch_w) + len(patch_mu)*len(patch_mu[1]) #+ len(patch_sigma)*len(patch_sigma[1])
    
        # Features extracted from a single patch
        # 3 + 6 + 6 --> repeated across 8 patches
        patch_features = list(patch_w) + list(patch_mu.ravel())+ list(patch_sigma.ravel())
        
        # concatenating from all patches
        total_features = total_features + patch_features
    # adding features from all patches to the face database 
    all_patch_data.append (total_features)

df_3D_face = pd.DataFrame({'Patch_Features':all_patch_data,'Classes':face_labels})                                      

# Encoding class labels 
le = preprocessing.LabelEncoder() 
df_3D_face['Labels']= le.fit_transform(df_3D_face['Classes']) 
 

In [ ]:
df_3D_face.to_pickle("all_face_features_spec.pkl")


#df_3D_face ['Classes'].unique()
df_3D_face 

In [7]:
def billbo (x,y):
    
    z = x*y;
    
    return z 

if __name__ == '__main__':
    
    alldata = []
    p= [3, 4, 5]
    q = [22, 33, 44]

    pool = Pool(8)

    data = [(p[i], q[i]) for i in range(3)]
     
    xx =pool.starmap(billbo,data)
    #feaVec = pool.map(square, range(0, 5))  

[(3, 22), (4, 33), (5, 44)]

In [ ]:
#Cut off 0.001
#fea_indx = np.array([ 0, 1, 2, 3, 5, 7, 15, 16, 17, 18, 20, 21, 22, 30, 31, 
#       32, 36, 45, 46, 47, 49, 51, 60, 61, 62, 63, 64, 65, 66, 67, 75, 76, 
#       77, 78, 80, 82, 90, 91, 94, 105, 106, 107, 108, 110, 111, 112, 113])

#Cut off 0.002
#fea_indx = np.array([ 0, 1, 2, 15, 16, 17, 30, 31, 32, 
#       36, 45, 46, 47, 61, 75, 77, 90, 91, 105, 106, 107])

#fea_indx =np.array([ 1, 2, 15, 30, 31 , 46, 77, 91, 105, 106])

# Load pickle file
df_3D_face = pd.read_pickle("all_face_features_lar.pkl")

xx = df_3D_face['Patch_Features'].to_list() 

data_mat = np.array (xx)

face_labels = df_3D_face['Labels'].to_list() 

n_labels = len(np.unique(face_labels)) 

#dataMatrix = data_mat   # [:, fea_indx]

data_matrix = data_mat# preprocessing.scale(data_mat)

print (len(data_matrix[1]))

df_3D_face

In [ ]:
df = df_3D_face.copy()

# Performing PCA on raw data 
pca = PCA(n_components=5)
pca_result = pca.fit_transform(data_matrix)
df['pca-one'] = pca_result[:,0]
df['pca-two'] = pca_result[:,1] 
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

plt.figure(figsize=(5,5))
sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="Classes",
    palette=sns.color_palette("hls", n_labels),
    data=df,
    legend="full",
    alpha=1.0
)

# Performing t-SNE on raw data
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=10, n_iter=300)
tsne_results = tsne.fit_transform(data_matrix)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

df['tsne-2d-one'] = tsne_results[:,0]
df['tsne-2d-two'] = tsne_results[:,1]

plt.figure(figsize=(5,5))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="Classes",
    palette=sns.color_palette("hls", n_labels),
    data=df,
    legend="full",
    alpha=1.0
)

# Performing t-SNE on PCA data 
tsne_results = tsne.fit_transform(pca_result[:,0:3])
#print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

df['tsne-2d-one'] = tsne_results[:,0]
df['tsne-2d-two'] = tsne_results[:,1]

plt.figure(figsize=(5,5))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="Classes",
    palette=sns.color_palette("hls", n_labels),
    data=df,
    legend="full",
    alpha=1.0
)



In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn import ( pipeline, preprocessing, linear_model)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.utils.multiclass import unique_labels
from matplotlib import pyplot as plt
#from sklearn.decomposition import PCA


#70% taining, 20% validating 10% testing
# 10x2 nested cross-validation 
# K-fold cross-validation - 1 fold -, 2-fold, average 
# 80% - 20%

print ('Number of features', len(data_matrix[1]))

# 10-fold cross-validation
# Each fold test - 35, train - 315
#Kaggle 
nfolds = 10
clf = GradientBoostingClassifier(n_estimators= 5,
                         #       class_weight = "balanced",
                                random_state=42#,cache_size=20000
                               # n_jobs=1
                               )

#pca = PCA ()   

pipe = pipeline.Pipeline([  ('scl', preprocessing.StandardScaler()),
                        #   ('pca', pca),
                            ('clf', clf) 
                           
                         
                         ])
    
param_grid = { # "pca__n_components": [5, 10, 15, 30, 40],
                      "clf__n_estimators":[10, 20, 30, 40, 50, 80,100],
              "clf__min_samples_split": [2, 5, 10, 15],
            "clf__learning_rate":[0.1, .02, 0.5],
             "clf__max_depth": [2, 5, 10, 15],
             }
    
    
        
kfold = StratifiedKFold(n_splits= nfolds, random_state= 42, shuffle=True)
    

 

fold_auc = []
testPredict = []
Stest = []

for k, (train, test) in enumerate (kfold.split(data_matrix,face_labels)):
         
        # Accumulating training fold data and labels
        trfold = [data_matrix [x] for x in train]
        trSen  = [face_labels[x] for x in train]
        
   
        # Accumulating test fold data and labels
        tsfold = [data_matrix[x] for x in test]
        tsSen = [face_labels[x] for x in test]
        
        
        
        gsModel = GridSearchCV(estimator=pipe, 
                                  param_grid = param_grid, n_jobs=1,
                                  scoring = 'accuracy', cv = 2)
         
       
        
        # training the model 
        trModel = gsModel.fit(trfold,trSen)
        
        bestModel = trModel.best_estimator_
        
        print (bestModel)
        
        testPred =  bestModel.predict(tsfold)
        
        ##### Predict Probability ############
        
        probas =  bestModel.predict_proba(tsfold)
        
        
        roc_auc =[]
        for t in range (len(np.unique(face_labels))):
            
            fpr, tpr, _ = roc_curve(tsSen,probas[:,t], pos_label=t)        
            roc_auc.append( auc(fpr,tpr))
            
        
        
        fold_auc.append (np.mean(roc_auc))
        
        
        testPredict.extend (testPred)
        Stest.extend (tsSen)
        
       # print ('Accuracy is:',accuracy_score( tsSen, testPred))
    
#    print(datetime.datetime.now().time())
print ('Mean 10-fold AUC', np.mean(fold_auc))
print ('Std 10-fold AUC', np.std(fold_auc))
    
#    return Stest, testPredict  

In [ ]:
def showAccuracy (fileName,Stest, testPredict):
    
    
    #Stest = tsSen
    cMatrix = confusion_matrix(Stest,testPredict)

    # Converts each value in the matrix into a percentage/float-value
    cMatrix = cMatrix.astype('float') / cMatrix.sum(axis=1)[:, np.newaxis]

    # Convert this list into a numpy array
    Sent_test = np.asarray(Stest, dtype=np.int) 


    fig, ax = plt.subplots(figsize=(6, 6))

    # The tick labels for the box-plot
    sent_label = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise'] 
    sent_label = np.asarray(sent_label, dtype=np.str)
    sent_label = sent_label[unique_labels(Stest, testPredict)]
    
    print (sent_label)

    # (cmap = plt.cm.Greens) represents the colors/shades used for the plot
    im = ax.imshow(cMatrix, interpolation='nearest', cmap=plt.cm.Greys)
    ax.figure.colorbar(im, ax=ax)
    
    # Configures the labels and tick marks of the box-plot


    ax.set(xticks=np.arange(cMatrix.shape[0] ),
       yticks=np.arange(cMatrix.shape[0]),
       
       # Label the ticks with their respective sentiment
       xticklabels=sent_label, yticklabels= sent_label,
       
       # Label the sides
       title='Confusion Matrix',
       ylabel='True Expression',
       xlabel='Predicted Expression')
   
    ax.set_ylim(6.5,-0.5) 
    # Rotates and aligns the tick labels to be upright and centered
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f'
    thresh = cMatrix.max() / 2.

    for i in range(cMatrix.shape[0]):
        for j in range(cMatrix.shape[0]):
            ax.text(j, i, format(cMatrix[i, j], fmt),
                ha="center", va="center",
                color="white" if cMatrix[i, j] > thresh else "black")

    # A report of something, not sure though...
  #  print(classification_report(Stest,testPredict))  

    # The overall accuracy of our test
    print('• Overall accuracy of the prediction/classifier model: ', accuracy_score(Sent_test, testPredict))

    # Prints the fancy box-plot
    #boxPlot.tight_layout()
   # ax.axis('off')

    fig.savefig(fileName, bbox_inches='tight', dpi=150)

showAccuracy ('confusion_01.pdf',Stest, testPredict)